# 蒼狼 AI V2.2 可觀測性分析平台 - Colab 快速啟動手冊

歡迎使用蒼狼 AI V2.2！本 Colab 筆記本旨在引導您在 Google Colaboratory 環境中快速設定並啟動應用程式的後端與前端服務。

**在開始之前，請確保您已仔細閱讀專案根目錄下的 `README.md` 文件，其中包含了關於架構、設定和依賴的詳細資訊。**

## Colab-步驟一：環境設定與授權

此步驟將掛載您的 Google Drive 並引導您設定必要的 API 金鑰和服務帳號憑證。

詳細的背景資訊、金鑰獲取方式以及安全提示，請參閱 [專案 README.md 中的「Colab-步驟一：環境設定與授權」章節](README.md#colab-步驟一環境設定與授權)。

**請執行下方的程式碼儲存格以開始。**

In [ ]:
# 掛載 Google Drive
from google.colab import drive
drive.mount('/content/drive') # 點擊連結並授權後，Drive 將掛載於 /content/drive

# 上傳服務帳號金鑰檔案 (方式一，推薦用於 Drive API)
# 如果您選擇使用 Colab Secrets 中的 GOOGLE_SERVICE_ACCOUNT_JSON_CONTENT (方式三)，則可以跳過或取消此檔案上傳步驟。
from google.colab import files
import os

print("\n步驟1.1 (可選): 上傳您的 Google Cloud 服務帳號 JSON 檔案 (例如 service_account.json)。")
print("如果您已在 Colab Secrets 中設定 GOOGLE_SERVICE_ACCOUNT_JSON_CONTENT，則此步驟不是必需的。")
try:
    # 清理先前可能已存在的 service_account.json，避免衝突
    if os.path.exists('service_account.json'): 
        os.remove('service_account.json') 
        print("已移除先前存在的 service_account.json。")

    uploaded = files.upload() # 觸發 Colab 檔案上傳介面

    if uploaded: # 如果使用者上傳了檔案
        original_filename = list(uploaded.keys())[0]
        # 為了統一，將上傳的檔案強制重新命名為 'service_account.json'
        if original_filename != 'service_account.json':
           os.rename(original_filename, 'service_account.json')
           print(f"檔案 '{original_filename}' 已上傳並重新命名為 'service_account.json'。")
        else:
           print(f"檔案 'service_account.json' 已成功上傳。")
        
        # 設定環境變數 GOOGLE_APPLICATION_CREDENTIALS 指向此檔案
        # 此環境變數由 Google Cloud 客戶端庫自動識別
        credentials_path = os.path.join('/content', 'service_account.json') # Colab 中的絕對路徑
        os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = credentials_path
        print(f"環境變數 GOOGLE_APPLICATION_CREDENTIALS 已成功設定為: {credentials_path}")
    else:
        print("\n使用者未上傳檔案。後續操作將依賴 Colab Secrets 中的 GOOGLE_SERVICE_ACCOUNT_JSON_CONTENT (如果已設定)。")
except Exception as e: # 捕獲檔案上傳過程中可能發生的任何錯誤 (例如使用者取消上傳)
    print(f"檔案上傳過程中發生錯誤 (可能是使用者取消了操作): {e}")
    print("後續操作將依賴 Colab Secrets 中的 GOOGLE_SERVICE_ACCOUNT_JSON_CONTENT (如果已設定)。")

print("\n提醒：請確保您已在 Colab 的「密鑰 (Secrets)」管理器中設定了 `COLAB_GOOGLE_API_KEY` (用於 Gemini 等 AI 服務)。")
print("同時，如果您未使用上傳檔案的方式，請確保已設定 `GOOGLE_SERVICE_ACCOUNT_JSON_CONTENT`。")

## Colab-步驟二：複製專案程式碼與安裝依賴

此步驟將從指定的 GitHub 倉庫下載最新的蒼狼 AI V2.2 專案程式碼，然後安裝所有必要的後端 Python 套件和前端 Node.js 模組。

詳細說明請參閱 [專案 README.md 中的「Colab-步驟二：複製專案程式碼與安裝依賴」章節](README.md#colab-步驟二複製專案程式碼與安裝依賴)。

**重要：請檢查下方的 `GIT_REPO_URL` 變數是否指向您期望的專案倉庫地址與分支。如果需要，請進行修改。**

In [ ]:
# 設定您的專案 GitHub 倉庫 URL
# TODO: 請確認此 URL 指向的是包含最新 V2.2 版本的正確倉庫！
GIT_REPO_URL = "https://github.com/hsp1234-web/wolf_ai_v2_2.git" 

# 定義專案在 Colab 環境中存放的路徑
PROJECT_PARENT_DIR = "/content/wolf_project" # Colab 中的父資料夾路徑
PROJECT_DIR_NAME = "wolf_ai_v2_2"       # 您的專案資料夾名稱
FULL_PROJECT_PATH = os.path.join(PROJECT_PARENT_DIR, PROJECT_DIR_NAME) # 完整的專案路徑

import os
import shutil # 用於移除資料夾

# 清理可能已存在的舊專案目錄，以確保每次都從 GitHub 獲取最新的程式碼
if os.path.exists(FULL_PROJECT_PATH):
    print(f"偵測到已存在的專案資料夾 '{FULL_PROJECT_PATH}'，將先移除以進行全新複製...")
    shutil.rmtree(FULL_PROJECT_PATH)
    print(f"舊專案資料夾 '{FULL_PROJECT_PATH}' 已成功移除。")

# 創建父資料夾 (如果它還不存在)
os.makedirs(PROJECT_PARENT_DIR, exist_ok=True)

print(f"開始從 {GIT_REPO_URL} 複製專案到 {FULL_PROJECT_PATH}...")
# 使用 --depth 1 選項進行淺層複製，這可以加快複製速度，因為它只獲取最新的提交歷史
clone_command = f"git clone --depth 1 {GIT_REPO_URL} {FULL_PROJECT_PATH}"
clone_status = os.system(clone_command) # 執行 git clone 命令

if clone_status != 0 or not os.path.isdir(FULL_PROJECT_PATH):
    print(f"錯誤：專案複製失敗。返回碼: {clone_status}。請檢查 GIT_REPO_URL ('{GIT_REPO_URL}') 是否正確以及您的網路連線。")
else:
    print(f"專案已成功複製到 '{FULL_PROJECT_PATH}' 目錄。")
    
    # 安裝後端依賴
    backend_requirements_path = os.path.join(FULL_PROJECT_PATH, 'backend', 'requirements.txt')
    if os.path.exists(backend_requirements_path):
        print("\n正在安裝後端 Python 依賴 (從 backend/requirements.txt)... 這可能需要幾分鐘時間。請耐心等候...")
        pip_install_command = f"pip install -r {backend_requirements_path} -q --no-warn-script-location"
        pip_status = os.system(pip_install_command) # -q 安靜模式，減少輸出
        if pip_status == 0: 
            print("後端 Python 依賴安裝完成。")
        else: 
            print(f"後端 Python 依賴安裝過程中發生錯誤 (返回碼: {pip_status})。請檢查 notebook 中的詳細錯誤日誌。")
    else:
        print(f"錯誤: 後端依賴定義檔 {backend_requirements_path} 未找到。跳過後端依賴安裝步驟。")
    
    # 安裝前端依賴
    frontend_dir = os.path.join(FULL_PROJECT_PATH, 'frontend')
    frontend_package_json_path = os.path.join(frontend_dir, 'package.json')
    if os.path.isdir(frontend_dir) and os.path.exists(frontend_package_json_path):
        print("\n正在安裝前端 Node.js 依賴 (從 frontend/package.json 使用 npm ci)... 這可能需要幾分鐘時間。請耐心等候...")
        # 使用 npm ci 可以確保安裝與 package-lock.json 一致的版本，通常比 npm install 更快更可靠
        # --prefix 選項讓 npm 在指定的目錄下執行
        # --verbose 提供更詳細的輸出，有助於排錯，但也可以移除以保持簡潔
        npm_ci_command = f"npm ci --prefix {frontend_dir} --verbose"
        npm_status = os.system(npm_ci_command)
        if npm_status == 0: 
            print("前端 Node.js 依賴安裝完成。")
        else: 
            print(f"前端 Node.js 依賴安裝過程中發生錯誤 (返回碼: {npm_status})。請檢查 notebook 中的詳細錯誤日誌。")
    else:
        print(f"錯誤：前端專案資料夾 '{frontend_dir}' 或其中的 'package.json' 檔案未找到。跳過前端依賴安裝步驟。")

print("\n--- 專案複製與依賴安裝階段完成 --- \n下一步是準備 Google Drive 資料夾和資料庫檔案。")

## Colab-步驟三：準備 Google Drive 資料夾與資料庫檔案

本專案使用 Google Drive 來持久化儲存資料庫檔案和報告文件。此步驟將協助您在 Drive 中建立必要的資料夾結構，並處理資料庫的同步。

詳細的資料夾結構說明和運作方式，請參閱 [專案 README.md 中的「Colab-步驟三：準備 Google Drive 資料夾與資料庫檔案」章節](README.md#colab-步驟三準備-google-drive-資料夾與資料庫檔案)。

**重要提示**：執行此儲存格後，它會嘗試創建資料夾並輸出 `WOLF_IN_FOLDER_ID` 和 `WOLF_PROCESSED_FOLDER_ID`。您必須將這兩個 ID **手動添加**到 Colab 的「密鑰 (Secrets)」管理器中，後端排程器才能正確監控 Drive 中的新報告。

所需密鑰名稱：
- `WOLF_IN_FOLDER_ID`
- `WOLF_PROCESSED_FOLDER_ID`

In [ ]:
import os
import shutil
from google.colab import auth # Colab 內建的授權模組
from googleapiclient.discovery import build # Google API Client Library
from googleapiclient.http import MediaIoBaseDownload # 用於下載 Drive 檔案
import io # 用於處理 BytesIO

drive_service_colab = None # 初始化 Drive API 服務物件為 None

print("正在初始化 Google Drive API 客戶端...")
try:
    # 根據環境變數決定使用哪種憑證初始化 Drive API 服務
    sa_file_path_env = os.getenv('GOOGLE_APPLICATION_CREDENTIALS')
    sa_json_content_env = os.getenv('GOOGLE_SERVICE_ACCOUNT_JSON_CONTENT')

    if sa_file_path_env and os.path.exists(sa_file_path_env):
        # 方式一：使用服務帳號檔案
        from google.oauth2.service_account import Credentials
        creds = Credentials.from_service_account_file(sa_file_path_env, scopes=['https://www.googleapis.com/auth/drive'])
        drive_service_colab = build('drive', 'v3', credentials=creds)
        print("成功：已使用上傳的服務帳號檔案 (GOOGLE_APPLICATION_CREDENTIALS) 初始化 Drive API。")
    elif sa_json_content_env:
        # 方式三：使用服務帳號 JSON 內容 (來自 Colab Secrets)
        import json
        from google.oauth2.service_account import Credentials
        creds_info = json.loads(sa_json_content_env)
        creds = Credentials.from_service_account_info(creds_info, scopes=['https://www.googleapis.com/auth/drive'])
        drive_service_colab = build('drive', 'v3', credentials=creds)
        print("成功：已使用 Colab Secret 'GOOGLE_SERVICE_ACCOUNT_JSON_CONTENT' 初始化 Drive API。")
    else:
        # 如果以上兩種服務帳號憑證都未提供，則回退到 Colab 使用者授權
        print("提示：未檢測到服務帳號憑證。將嘗試使用 Colab 當前登入使用者的授權來操作 Google Drive。")
        print("這通常需要您在彈出視窗中手動確認授權。")
        auth.authenticate_user() # 觸發 Colab 使用者授權流程
        drive_service_colab = build('drive', 'v3') # 使用已授權使用者的憑證
        print("成功：Colab 使用者已授權，可以使用其權限操作 Drive API。")
except Exception as e:
    print(f"錯誤：初始化 Google Drive API 客戶端時發生嚴重錯誤: {e}")
    print("後續的 Drive 資料夾創建和檔案下載操作可能失敗。請檢查您的憑證設定。")
    drive_service_colab = None # 確保出錯時 drive_service_colab 為 None

# 在 Google Drive 中定義的資料夾名稱
DRIVE_BASE_DIR_NAME = "wolf_AI_data"  # 專案在 Drive 中的主資料夾名稱
DRIVE_INBOX_NAME = "wolf_in"         # 存放待處理報告的資料夾名稱
DRIVE_PROCESSED_NAME = "processed"   # 存放已處理報告的資料夾名稱

# Colab 本地資料路徑 (相對於 FULL_PROJECT_PATH)
COLAB_LOCAL_DATA_PATH = os.path.join(FULL_PROJECT_PATH, 'backend', 'data') 
os.makedirs(COLAB_LOCAL_DATA_PATH, exist_ok=True) # 確保本地 data 資料夾存在

# 輔助函數：獲取或創建 Google Drive 資料夾，並返回其 ID
def get_or_create_drive_folder(service, folder_name, parent_id=None):
    # 建立查詢語句，尋找指定名稱和類型的資料夾
    query = f"name='{folder_name}' and mimeType='application/vnd.google-apps.folder'"
    if parent_id: # 如果提供了父資料夾 ID，則在父資料夾中尋找
        query += f" and '{parent_id}' in parents"
    else: # 否則在根目錄下尋找
        query += " and 'root' in parents"
    query += " and trashed=false" # 確保資料夾未被移入回收站
    
    try:
        response = service.files().list(q=query, spaces='drive', fields='files(id, name)').execute()
        folders = response.get('files', [])
        if folders: # 如果找到資料夾
            print(f"Drive 資料夾 '{folder_name}' (在父ID: {parent_id or 'root'}) 已存在，其 ID 為: {folders[0].get('id')}")
            return folders[0].get('id')
        else: # 如果未找到資料夾，則創建它
            print(f"Drive 資料夾 '{folder_name}' (在父ID: {parent_id or 'root'}) 不存在，正在嘗試創建...")
            file_metadata = {'name': folder_name, 'mimeType': 'application/vnd.google-apps.folder'}
            if parent_id:
                file_metadata['parents'] = [parent_id]
            folder = service.files().create(body=file_metadata, fields='id').execute()
            print(f"Drive 資料夾 '{folder_name}' 創建成功，其 ID 為: {folder.get('id')}")
            return folder.get('id')
    except Exception as error:
        print(f"操作 Drive 資料夾 '{folder_name}' 時發生錯誤: {error}")
        return None

wolf_in_folder_id_val = None       # 初始化 WOLF_IN_FOLDER_ID 的值
wolf_processed_folder_id_val = None # 初始化 WOLF_PROCESSED_FOLDER_ID 的值

if drive_service_colab: # 只有 Drive API 服務成功初始化才執行以下操作
    # 1. 獲取或創建基礎資料夾 wolf_AI_data
    drive_base_folder_id = get_or_create_drive_folder(drive_service_colab, DRIVE_BASE_DIR_NAME)
    
    if drive_base_folder_id: # 如果成功獲取/創建基礎資料夾
        # 2. 在基礎資料夾下獲取或創建 wolf_in 資料夾
        wolf_in_folder_id_val = get_or_create_drive_folder(drive_service_colab, DRIVE_INBOX_NAME, drive_base_folder_id)
        if wolf_in_folder_id_val:
            # 3. 在 wolf_in 資料夾下獲取或創建 processed 資料夾
            wolf_processed_folder_id_val = get_or_create_drive_folder(drive_service_colab, DRIVE_PROCESSED_NAME, wolf_in_folder_id_val)
        
        # 4. 檢查並下載資料庫檔案 (reports.sqlite, prompts.sqlite)
        for db_filename in ["reports.sqlite", "prompts.sqlite"]:
            colab_local_db_file_path = os.path.join(COLAB_LOCAL_DATA_PATH, db_filename)
            # 查詢 Drive 中是否存在此資料庫檔案 (在 drive_base_folder_id 下)
            query_db = f"name='{db_filename}' and '{drive_base_folder_id}' in parents and trashed=false"
            try:
                response_db = drive_service_colab.files().list(q=query_db, fields='files(id, name, size)').execute()
                drive_db_files = response_db.get('files', [])
                if drive_db_files:
                    drive_db_file_id = drive_db_files[0]['id']
                    drive_db_file_size = drive_db_files[0].get('size', '未知大小')
                    print(f"在 Drive 的 '{DRIVE_BASE_DIR_NAME}' 資料夾中找到資料庫檔案 '{db_filename}' (ID: {drive_db_file_id}, 大小: {drive_db_file_size} bytes)。")
                    print(f"正在嘗試將其下載到 Colab 本地路徑: {colab_local_db_file_path}...")
                    
                    request = drive_service_colab.files().get_media(fileId=drive_db_file_id)
                    fh = io.BytesIO() # 創建一個 BytesIO 物件作為暫存
                    downloader = MediaIoBaseDownload(fh, request) # 創建下載器
                    done = False
                    while not done:
                        status, done = downloader.next_chunk()
                        if status:
                             print(f"  下載進度: {int(status.progress() * 100)}%")
                    fh.seek(0) # 移動到 BytesIO 物件的開頭
                    with open(colab_local_db_file_path, 'wb') as f_out:
                        shutil.copyfileobj(fh, f_out) # 將下載的內容寫入本地檔案
                    print(f"資料庫檔案 '{db_filename}' 已成功下載到本地。")
                else:
                    print(f"提示：在 Drive 的 '{DRIVE_BASE_DIR_NAME}' 資料夾中未找到資料庫檔案 '{db_filename}'。如果這是首次執行，後端應用程式將在本地自動創建新的空資料庫。")
            except Exception as error:
                print(f"在 Drive 中搜尋或下載資料庫檔案 '{db_filename}' 時發生錯誤: {error}")
else:
    print("警告：Google Drive API 服務未能成功初始化。無法自動創建 Drive 資料夾或下載/檢查資料庫檔案。")
    print("請確保您已正確設定服務帳號憑證 (透過上傳檔案或 Colab Secrets)，或者已對 Colab 使用者授權。")
    print("您可能需要手動在 Google Drive 中創建 '{DRIVE_BASE_DIR_NAME}', '{DRIVE_INBOX_NAME}', '{DRIVE_PROCESSED_NAME}' 資料夾結構，")
    print("並將相應的 Folder ID 手動填入 Colab Secrets。")

print("\n" + "-"*20 + " Google Drive 資料夾和資料庫準備總結 " + "-"*20)
if wolf_in_folder_id_val:
    print(f"🟢 WOLF_IN_FOLDER_ID: {wolf_in_folder_id_val}  <-- 重要：請將此 ID 設定到 Colab Secrets 中! (如果尚未設定)")
else:
    print("🔴 WOLF_IN_FOLDER_ID: 未能自動獲取或創建。請手動處理並設定到 Colab Secrets。")

if wolf_processed_folder_id_val:
    print(f"🟢 WOLF_PROCESSED_FOLDER_ID: {wolf_processed_folder_id_val}  <-- 重要：請將此 ID 設定到 Colab Secrets 中! (如果尚未設定)")
else:
    print("🔴 WOLF_PROCESSED_FOLDER_ID: 未能自動獲取或創建。請手動處理並設定到 Colab Secrets。")

if not drive_service_colab or not drive_base_folder_id:
    print("🔴 資料庫檔案: 由於 Drive服務/基礎資料夾問題，未檢查或下載。應用程式將嘗試在本地創建新資料庫。")

## Colab-步驟四：啟動應用程式服務與健康度偵測

此步驟將啟動後端 FastAPI 伺服器和前端 Next.js 開發伺服器，並執行健康度檢查以確認服務是否成功運行。

詳細的啟動參數和日誌說明，請參閱 [專案 README.md 中的「Colab-步驟四：啟動應用程式服務與健康度偵測」章節](README.md#colab-步驟四啟動應用程式服務與健康度偵測)。

**在執行服務啟動之前，下方新增了一個儲存格，用於檢查必要的 Colab Secrets 是否已設定。**

In [ ]:
# Colab Secrets 環境變數檢查
import os

print("正在檢查必要的 Colab Secrets 設定情況...")
required_secrets_to_check = {
    'COLAB_GOOGLE_API_KEY': False,
    'WOLF_IN_FOLDER_ID': False,
    'WOLF_PROCESSED_FOLDER_ID': False,
    'GOOGLE_CREDENTIALS_CHECK': False # 特殊檢查，代表 GOOGLE_APPLICATION_CREDENTIALS 或 GOOGLE_SERVICE_ACCOUNT_JSON_CONTENT
}

colab_google_api_key = os.getenv('COLAB_GOOGLE_API_KEY')
wolf_in_folder_id = os.getenv('WOLF_IN_FOLDER_ID')
wolf_processed_folder_id = os.getenv('WOLF_PROCESSED_FOLDER_ID')
google_app_creds = os.getenv('GOOGLE_APPLICATION_CREDENTIALS')
google_sa_json_content = os.getenv('GOOGLE_SERVICE_ACCOUNT_JSON_CONTENT')

if colab_google_api_key:
    required_secrets_to_check['COLAB_GOOGLE_API_KEY'] = True
    print("  ✅ COLAB_GOOGLE_API_KEY 已設定。")
else:
    print("  ❌ COLAB_GOOGLE_API_KEY 未設定或為空值！")

if wolf_in_folder_id:
    required_secrets_to_check['WOLF_IN_FOLDER_ID'] = True
    print("  ✅ WOLF_IN_FOLDER_ID 已設定。")
else:
    print("  ❌ WOLF_IN_FOLDER_ID 未設定或為空值！")

if wolf_processed_folder_id:
    required_secrets_to_check['WOLF_PROCESSED_FOLDER_ID'] = True
    print("  ✅ WOLF_PROCESSED_FOLDER_ID 已設定。")
else:
    print("  ❌ WOLF_PROCESSED_FOLDER_ID 未設定或為空值！")

if (google_app_creds and os.path.exists(google_app_creds)) or google_sa_json_content:
    required_secrets_to_check['GOOGLE_CREDENTIALS_CHECK'] = True
    if google_app_creds and os.path.exists(google_app_creds):
        print(f"  ✅ Google 服務帳號憑證已透過檔案 '{google_app_creds}' 設定。")
    if google_sa_json_content:
        print("  ✅ Google 服務帳號憑證已透過 Colab Secret 'GOOGLE_SERVICE_ACCOUNT_JSON_CONTENT' 設定。")
    if (google_app_creds and os.path.exists(google_app_creds)) and google_sa_json_content:
        print("     (提示：同時偵測到檔案和 JSON 內容憑證，後端將優先使用其一，通常是 GOOGLE_SERVICE_ACCOUNT_JSON_CONTENT。)")
else:
    print("  ❌ Google 服務帳號憑證 (GOOGLE_APPLICATION_CREDENTIALS 檔案路徑或 GOOGLE_SERVICE_ACCOUNT_JSON_CONTENT) 未有效設定！")

all_set = all(required_secrets_to_check.values())
if not all_set:
    print("\n⚠️ **警告：一個或多個必要的 Colab Secret 未被正確設定！**")
    print("請返回前面的步驟，或直接檢查 Colab 的「密鑰 (Secrets)」管理器，確保以下項目已正確設定:")
    if not required_secrets_to_check['COLAB_GOOGLE_API_KEY']: print("    - COLAB_GOOGLE_API_KEY (用於 AI 服務)")
    if not required_secrets_to_check['WOLF_IN_FOLDER_ID']: print("    - WOLF_IN_FOLDER_ID (Google Drive 輸入資料夾 ID)")
    if not required_secrets_to_check['WOLF_PROCESSED_FOLDER_ID']: print("    - WOLF_PROCESSED_FOLDER_ID (Google Drive 已處理資料夾 ID)")
    if not required_secrets_to_check['GOOGLE_CREDENTIALS_CHECK']: print("    - GOOGLE_APPLICATION_CREDENTIALS (服務帳號金鑰檔案路徑) 或 GOOGLE_SERVICE_ACCOUNT_JSON_CONTENT (服務帳號金鑰 JSON 內容)")
    print("缺少這些關鍵設定可能會導致應用程式啟動失敗或核心功能 (如 Drive 存取、AI 分析) 無法正常運作。")
else:
    print("\n✅ 所有必要的 Colab Secrets 看起來都已設定。可以繼續執行下一步以啟動應用程式。")


In [ ]:
import os
import time
import subprocess # 用於執行背景命令
import requests   # 用於執行健康檢查請求
from IPython.display import display, HTML # 用於在 Colab 中顯示格式化的 HTML 輸出
import urllib.parse # 用於 URL 解析 (如果需要)

print("--- 第四步：準備啟動後端與前端應用程式服務 ---")

# 檢查 FULL_PROJECT_PATH 是否已在先前儲存格中定義且有效
# 如果此儲存格作為獨立部分執行，可能需要重新定義這些路徑變數
if 'FULL_PROJECT_PATH' not in locals() or not os.path.isdir(FULL_PROJECT_PATH):
    print("警告：變數 FULL_PROJECT_PATH 未定義或指向的不是一個有效目錄。")
    print("將嘗試使用預設的專案路徑：'/content/wolf_project/wolf_ai_v2_2'。")
    PROJECT_PARENT_DIR = "/content/wolf_project"
    PROJECT_DIR_NAME = "wolf_ai_v2_2"
    FULL_PROJECT_PATH = os.path.join(PROJECT_PARENT_DIR, PROJECT_DIR_NAME)
    if not os.path.isdir(FULL_PROJECT_PATH):
        print(f"錯誤：預設專案路徑 {FULL_PROJECT_PATH} 也不存在或無效。")
        print("請確保您已成功執行了「Colab-步驟二：複製專案程式碼與安裝依賴」的儲存格。")
        raise SystemExit("關鍵專案路徑不存在，無法繼續啟動服務。") # 在 Colab 中，這會停止執行

# 定義相關路徑和端口號
COLAB_LOCAL_DATA_PATH = os.path.join(FULL_PROJECT_PATH, 'backend', 'data') # Colab 本地資料庫的存放路徑
BACKEND_LOG_FILE = os.path.join(FULL_PROJECT_PATH, 'backend_service.log') # 後端服務日誌檔案
FRONTEND_LOG_FILE = os.path.join(FULL_PROJECT_PATH, 'frontend_service.log') # 前端服務日誌檔案
BACKEND_SERVICE_PORT = 8000 # 後端 FastAPI 服務運行的端口
FRONTEND_SERVICE_PORT = 3000 # 前端 Next.js 服務運行的端口

# 清理舊的日誌檔案 (如果存在)
for log_file_path in [BACKEND_LOG_FILE, FRONTEND_LOG_FILE]:
    if os.path.exists(log_file_path): 
        os.remove(log_file_path)
        print(f"已移除舊的日誌檔案: {log_file_path}")

# 設定資料庫檔案的實際路徑環境變數，供後端 DataAccessLayer 使用
# 後端 main.py 應設計為優先使用這些環境變數來確定資料庫檔案位置
os.environ['REPORTS_DB_PATH'] = os.path.join(COLAB_LOCAL_DATA_PATH, 'reports.sqlite')
os.environ['PROMPTS_DB_PATH'] = os.path.join(COLAB_LOCAL_DATA_PATH, 'prompts.sqlite')
print(f"環境變數 REPORTS_DB_PATH 已設定為: {os.environ['REPORTS_DB_PATH']}")
print(f"環境變數 PROMPTS_DB_PATH 已設定為: {os.environ['PROMPTS_DB_PATH']}")

# 注意：Colab Secrets (如 COLAB_GOOGLE_API_KEY, WOLF_IN_FOLDER_ID 等) 的檢查已移至上一個儲存格
# 此處假設使用者已執行過該檢查並已設定好必要的 Secrets

# 啟動後端 FastAPI 服務
backend_project_dir = os.path.join(FULL_PROJECT_PATH, 'backend')
# 使用 nohup 和 & 將服務放到背景執行，並將標準輸出和錯誤輸出重定向到日誌檔案
backend_start_command = f"nohup python main.py > {BACKEND_LOG_FILE} 2>&1 &"
print(f"\n準備在目錄 {backend_project_dir} 中啟動後端 FastAPI 服務...")
subprocess.Popen(backend_start_command, shell=True, cwd=backend_project_dir, executable='/bin/bash')
print(f"後端服務已在背景啟動。詳細日誌請查看: {BACKEND_LOG_FILE}")

# 啟動前端 Next.js 服務
frontend_project_dir = os.path.join(FULL_PROJECT_PATH, 'frontend')
# -p 指定端口，-H 0.0.0.0 使其監聽所有接口 (Colab 需要)
frontend_start_command = f"nohup npm run dev -- -p {FRONTEND_SERVICE_PORT} -H 0.0.0.0 > {FRONTEND_LOG_FILE} 2>&1 &"
print(f"\n準備在目錄 {frontend_project_dir} 中啟動前端 Next.js 服務...")
subprocess.Popen(frontend_start_command, shell=True, cwd=frontend_project_dir, executable='/bin/bash')
print(f"前端服務已在背景啟動。詳細日誌請查看: {FRONTEND_LOG_FILE}")

# 健康度偵測迴圈
print("\n" + "-"*15 + " 健康度偵測 (Health Check) - 請耐心等待約 1-2 分鐘 " + "-"*15)
max_wait_duration_seconds = 120  # 最大等待時間 (秒)
check_interval_seconds = 10    # 每次檢查之間的間隔時間 (秒)
start_check_time = time.time()
backend_service_ready, frontend_service_ready = False, False

while time.time() - start_check_time < max_wait_duration_seconds:
    # 檢查後端服務狀態
    if not backend_service_ready:
        try:
            response_backend = requests.get(f"http://localhost:{BACKEND_SERVICE_PORT}/api/health", timeout=3)
            if response_backend.status_code == 200 and response_backend.json().get('status') == 'OK':
                backend_service_ready = True
                print(f"[{time.strftime('%H:%M:%S')}] ✅ 後端服務 READY. 狀態: {response_backend.json()}")
            else:
                print(f"[{time.strftime('%H:%M:%S')}] ⏳ 後端服務未就緒 (HTTP {response_backend.status_code}) - {response_backend.text[:100]}")
        except requests.RequestException as e:
            print(f"[{time.strftime('%H:%M:%S')}] ⏳ 後端服務未就緒 (連接失敗: {str(e)[:100]})")
    
    # 檢查前端服務狀態 (通常前端開發伺服器在根路徑 '/' 返回 200 OK)
    if not frontend_service_ready:
        try:
            response_frontend = requests.get(f"http://localhost:{FRONTEND_SERVICE_PORT}", timeout=3)
            if response_frontend.status_code == 200:
                frontend_service_ready = True
                print(f"[{time.strftime('%H:%M:%S')}] ✅ 前端服務 READY.")
            else:
                print(f"[{time.strftime('%H:%M:%S')}] ⏳ 前端服務未就緒 (HTTP {response_frontend.status_code})")
        except requests.RequestException as e:
            print(f"[{time.strftime('%H:%M:%S')}] ⏳ 前端服務未就緒 (連接失敗: {str(e)[:100]})")

    if backend_service_ready and frontend_service_ready: 
        print("\n🎉 太棒了！後端和前端服務均已成功啟動並通過健康檢查！")
        break # 跳出迴圈
    
    if time.time() - start_check_time >= max_wait_duration_seconds: 
        print("\n⏱️ 服務啟動超時。請檢查日誌檔案以了解詳細資訊。")
        break # 超時也跳出迴圈
        
    print(f"   ...等待 {check_interval_seconds} 秒後重試...")
    time.sleep(check_interval_seconds)

# 最終打印訪問資訊
if backend_service_ready and frontend_service_ready:
    # Colab 通常會將監聽 0.0.0.0 的端口代理到一個公開的 googleusercontent.com URL
    # 我們可以嘗試使用 Colab 的 JavaScript API 來獲取這個 URL，但更可靠的方式是引導使用者查看 Colab 的輸出
    # (因為 eval_ betekent JavaScript 可能不總是按預期工作或被禁用)
    
    # 嘗試獲取 Colab 的代理 URL (可能不總是成功)
    colab_proxy_url = None
    try:
        # 這段 JavaScript 嘗試獲取 Colab 為指定端口創建的代理 URL
        # 注意: 這依賴 Colab 的內部實現，未來可能會有變化
        js_code = f"google.colab.kernel. τότεxt.manager.getResults('/tune/fine-grained-proxy/{FRONTEND_SERVICE_PORT}/').then(function(response) {{ return response.data['proxy_url']; }})"
        # colab_proxy_url = display(Javascript(js_code), display_id=True) # display_id 方式較複雜
        # 簡化處理：直接提示使用者，並顯示一個基於 localhost 的備用連結
        pass # 由於直接獲取 URL 不穩定，改為引導使用者
    except Exception as e_js:
        print(f"(嘗試自動獲取 Colab 公開 URL 時發生錯誤: {e_js})")
    
    border = "="*60
    print(f"\n{border}")
    display(HTML("<h2>✅ 應用程式已就緒，可以開始使用了！</h2>"))
    print(f"🔗 **前端應用程式訪問網址 (Colab Proxy URL):**")
    print(f"   請在**本儲存格的輸出區域上方**尋找由 Colab 自動生成的、形如 `https://*.googleusercontent.com/` 的公開連結。")
    print(f"   這個連結指向您的前端應用程式 (端口 {FRONTEND_SERVICE_PORT})。")
    print("   如果 Colab 沒有自動顯示連結，您可能需要檢查 Colab 的相關設定或網路情況。")
    # 提供一個備用連結，雖然它可能只能在特定 Colab 配置下運作
    # print(f"   備用 (可能僅限部分環境): http://localhost:{FRONTEND_SERVICE_PORT}") 
    display(HTML(f"<p style='font-size:1.1em;'>點擊 Colab 在此儲存格輸出上方提供的 <code>https://....googleusercontent.com/</code> 公開連結即可訪問。</p>"))
    print(f"{border}\n")
else:
    print("\n❌ 很抱歉，一個或多個服務未能成功啟動。請仔細檢查上方的健康度偵測日誌以及服務日誌檔案獲取詳細錯誤信息：")

print(f"  後端服務日誌檔案位於: {BACKEND_LOG_FILE}")
print(f"  前端服務日誌檔案位於: {FRONTEND_LOG_FILE}")
print("  您可以點擊左側資料夾圖示，導航到相應路徑查看日誌內容。")

print("\n--- Colab 腳本執行完畢 ---")